In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 59.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
! pip install tqdm

In [3]:
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm

#Filtering the name entities that were incorrected labeled with the spaCy library

In [ ]:

df = pd.read_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/entities_frequency.csv")


ner_pipeline = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", tokenizer="mrm8488/bert-spanish-cased-finetuned-ner", grouped_entities=True)


def is_correctly_tagged(entity):
    try:
        results = ner_pipeline(entity)
        for r in results:
            if r['entity_group'] in ['LOC', 'ORG']:
                return True
    except:
        pass
    return False


tqdm.pandas()
df['correctly_tagged'] = df['entity'].progress_apply(is_correctly_tagged)
filtered_df = df[df['correctly_tagged'] == True].drop(columns=['correctly_tagged'])


filtered_df.to_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/filtered_entities.csv", index=False)
print(filtered_df)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


  0%|          | 0/16145 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


             entity type  frequency
1            Madrid  LOC       1911
3            España  LOC       1055
4           Dolores  LOC        898
9             Santa  LOC        486
10          Sevilla  LOC        389
...             ...  ...        ...
16124   Casa-Malaña  LOC          1
16129      Amazonas  LOC          1
16134  Puerto-Arcas  LOC          1
16137     Montceaux  LOC          1
16140         Arcas  LOC          1

[3691 rows x 3 columns]


In [ ]:
import pandas as pd
import re

#Sentiment Analysis of a selected group of exotic locations

In [ ]:
import os
import glob
import random
import xml.etree.ElementTree as ET
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

sentiment_analyzer = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

route = "/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Spanish_ELTec_Corpus"
files = glob.glob(os.path.join(route, "*_annotated.xml"))

# Key words
key_words = [
    "Francia", "Inglaterra", "París", "Roma", "Italia", "Flandes", "Londres",
    "América", "Portugal", "Nápoles", "Maya", "Fátima", "Alemania", "Rajatul-laj",
    "Occidente", "Austria", "África", "Túnez", "Birmingham", "Habana", "Lisboa",
    "Cuba", "Bruselas", "Méjico", "Biarritz", "Indias", "Asia", "Egipto",
    "Constantinopla", "Holanda", "Damasco", "Filipinas", "India", "Bretaña",
    "Rusia", "Grecia", "Escocia"
]

def extract_tokens(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    tokens = []
    for w in root.findall('.//tei:w', ns):
        text = w.text
        if text:
            tokens.append({
                "text": text,
                "pos": w.attrib.get("pos", ""),
                "lemma": w.attrib.get("lemma", ""),
                "ner": w.attrib.get("ner", "")
            })
    return tokens

# Obtain the year of publication in order to compare periods
def get_year(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    date_node = root.find('.//tei:sourceDesc//tei:bibl[@type="firstEdition"]/tei:date', ns)
    if date_node is not None and date_node.text and date_node.text.strip().isdigit():
        return int(date_node.text.strip())
    return None

# Classify the years in 2 decade periods
def classify_year(year):
    if year is None:
        return "unknown"
    if 1840 <= year <= 1859:
        return "1840-1859"
    elif 1860 <= year <= 1879:
        return "1860-1879"
    elif 1880 <= year <= 1899:
        return "1880-1899"
    elif 1900 <= year <= 1920:
        return "1900-1920"
    return "Out of range"

# Get windows of text
def windows(tokens, key_word, file_, period, window=100):
    windows = []
    for i, token in enumerate(tokens):
        if token["text"] == key_word:
            start = max(i - window, 0)
            end = min(i + window + 1, len(tokens))
            context = " ".join([tok["text"] for tok in tokens[start:end]])
            windows.append({
                "key_word": key_word,
                "text_window": context,
                "file": os.path.basename(file_),
                "period": period
            })
    return windows

# Get all the windows that contain the key_word
all_windows = []

for file_ in tqdm(files, desc="Processing all files"):
    year = get_year(file_)
    period = classify_year(year)
    tokens = extract_tokens(file_)

    for word in key_words:
        new = windows(tokens, word, file_, period)
        all_windows.extend(new)

# Sentiment Analysis in all the windows that have one key_word on the text
for window in tqdm(all_windows, desc="Analyzing sentiments"):
    try:
        result = sentiment_analyzer(window["text_window"][:512])[0]
        window["sentiment"] = result["label"]
        window["confidence_score"] = result["score"]
    except Exception as e:
        window["sentiment"] = "error"
        window["confidence_score"] = 0.0


df = pd.DataFrame(all_windows)

# Select all windows that mention a key word (country) with the best confidence score
df_top30 = (
    df.sort_values(by=["key_word", "confidence_score"], ascending=[True, False])
      .groupby("key_word")
      .head(30)
      .reset_index(drop=True)
)

df_top30.to_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/data/sentiment.csv", index=False)

print("Df saved.")

Device set to use cuda:0
Analyzing sentiments: 100%|██████████| 4992/4992 [00:30<00:00, 161.12it/s]

Df saved.


In [ ]:
df_top30.head()

,key_word,text_window,file,period,sentiment,confidence_score
0,Alemania,oscuros que los de su hermana Oh señora qué he...,SPA2022_Sinues_ElAlmaEnferma_annotated.xml,1860-1879,Very Positive,0.830967
1,Alemania,al armario Su tío le detuvo —No hace falta chi...,SPA3022_PalacioValdes_LaEspuma_annotated.xml,1880-1899,Very Negative,0.787647
2,Alemania,desacato a su persona u otro crimen análogo qu...,SPA4025_Coloma_Jeromin_annotated.xml,1900-1920,Very Negative,0.751216
3,Alemania,nombre familiar del presidente Kruger a los en...,SPA4017_Maetzu_LaGuerraDelTransvaal_annotated.xml,1900-1920,Very Positive,0.714869
4,Alemania,á los desesperados esfuerzos de un hombre No h...,SPA1018_Andueza_DonFelipe_annotated.xml,1840-1859,Very Negative,0.708247


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/data/sentiment.csv")

#Analysis of the results

In [5]:
conteo = df.groupby('key_word')['sentiment'].value_counts().unstack(fill_value=0)
print(conteo)

sentiment       Negative  Neutral  Positive  Very Negative  Very Positive
key_word                                                                 
Alemania               0        1         3             15             11
América                0        1         0             13             16
Asia                   0        2         3             14             11
Austria                0        1         0             16             13
Biarritz               0        1         1              9             19
Birmingham             2        3         2             16              7
Bretaña                1        2         7             11              9
Bruselas               0        0         1             20              9
Constantinopla         0        1         3             19              7
Cuba                   0        1         1             14             14
Damasco                0        0         5              2              1
Egipto                 0        2     

# Collapsing the sentiment labels

## Selecting only the countries by interest (Asia, America, Egypt, Philippines and Cuba)

In [6]:
import re

df['file'] = df['file'].astype(str)

# Modifying the incorrect labeled files
periods = {
    'SPA3029_Blanco_Cercado_annotated.xml': '1880-1899',
    'SPA2021_RafaelDelCastillo_LosCaballerosDelAmor_annotated.xml': '1860-1879',
    'SPA3002_LeopoldoAlas_LaRegenta_annotated.xml': '1900-1920',
    'SPA1022_GomezDeAvellaneda_DosMujeres_annotated.xml': '1840-1859'
}


for file, period in periods.items():
    df.loc[df['file'] == file, 'period'] = period

# Collapsing the labels into 2 and droping the Neutral instances
reduced_sentiment = {
    'Very Negative': 'Negative',
    'Very Positive': 'Positive',
    'Neutral': None
}

df['sentiment'] = df['sentiment'].map(reduced_sentiment)
df = df.dropna(subset=['sentiment'])

In [7]:
df.shape

(945, 6)

In [8]:
# Selecting only some countries to analyse
countries = ['Cuba', 'Asia', 'América', 'Egipto', 'Filipinas']
regex = '|'.join(map(re.escape, countries))
df = df[df['key_word'].str.contains(regex, case=False, na=False)]


### Average of the confidence score per sentiment and Country

In [9]:
df.groupby(['key_word', 'sentiment'])['confidence_score'].mean()

key_word   sentiment
América    Negative     0.714441
           Positive     0.694614
Asia       Negative     0.579091
           Positive     0.557891
Cuba       Negative     0.587955
           Positive     0.624881
Egipto     Negative     0.593240
           Positive     0.592586
Filipinas  Negative     0.560906
           Positive     0.542829
Name: confidence_score, dtype: float64

### Number of sentiment labels per country

In [ ]:
df.groupby('key_word')['sentiment'].value_counts().unstack(fill_value=0)

sentiment,Negative,Positive
key_word,,
América,13,16
Asia,14,11
Cuba,14,14
Egipto,7,16
Filipinas,11,12


### Number of instances per country and period

In [ ]:
df.groupby(['key_word'])['period'].value_counts()

key_word   period   
América    1860-1879    14
           1880-1899     6
           1900-1920     6
           1840-1859     3
Asia       1900-1920    10
           1860-1879     9
           1880-1899     4
           1840-1859     2
Cuba       1880-1899    11
           1900-1920     9
           1860-1879     5
           1840-1859     3
Egipto     1840-1859    12
           1880-1899     5
           1900-1920     4
           1860-1879     2
Filipinas  1880-1899    12
           1860-1879     7
           1900-1920     3
           1840-1859     1
Name: count, dtype: int64

### Number of files per country

In [11]:
df.groupby(['key_word'])['file'].nunique()

,file
key_word,
América,17
Asia,15
Cuba,16
Egipto,13
Filipinas,9


# Pos tagging all windows to get cleaner wordClouds

In [ ]:
import os
import spacy
from collections import defaultdict


nlp = spacy.load("es_core_news_sm")


os.makedirs("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Results", exist_ok=True)

# Dictionary to save the words by group
words_by_group = defaultdict(list)


for _, row in df.iterrows():
    text = row['text_window']
    sentiment = row['sentiment']
    key_word = row['key_word']
    period = row['period']

    doc = nlp(text)

    # POS tag filtering
    filtered_words = [
        token.lemma_.lower() for token in doc
        if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV'] and token.pos_ != 'PROPN'
        and not token.is_stop and token.is_alpha
    ]

    #Saving the files
    file_path = f"{key_word}__{sentiment}_{period}"
    words_by_group[file_path].extend(filtered_words)


for file_path, words in words_by_group.items():
    clean_file_path = re.sub(r'[^\w\-_]', '_', file_path)
    name_file = f"/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Results/{clean_file_path}.txt"

    with open(name_file, 'w', encoding='utf-8') as f:
        f.write(" ".join(words))


# Cleaning with tf-idf

In [ ]:
import os

input_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Results'


words_by_file = {}

# Leer todos los archivos .txt
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            words = f.read().split()
            words_by_file[filename] = set(words)

# Obtaining the common words for all of the files
if words_by_file:
    common_words = set.intersection(*words_by_file.values())
else:
    common_words = set()

# Filtering the common words across all files
for filename, words in words_by_file.items():
    filtered_words = words - common_words
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(" ".join(filtered_words))

# WordClouds

In [ ]:
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt


output_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Results/wordCloud'
os.makedirs(output_dir, exist_ok=True)


input_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/Results'

# Get all .txt file of sentiments
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_dir, filename)


        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()

        # WordCloud Creatin
        wc = WordCloud(
            width=800, height=400,
            background_color='white',
            colormap='viridis'
        ).generate(text)

        # Save Image
        output_path = os.path.join(output_dir, filename.replace('.txt', '.png'))
        wc.to_file(output_path)


# Analysis of the results

We selected some interesting words to perform a qualitative analysis of the resulting wordClouds

### "Familia" -- Negative

In [ ]:
df_neg_familia = df[
    (df['sentiment'] == 'Negative') &
    (df['text_window'].str.contains(r'\bfamilia\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Negative' and that contain the word 'familia': {len(df_neg_familia)}")


for i, text in enumerate(df_neg_familia['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")


Number of instances with sentiment 'Negative' and that contain the word 'familia': 6

Text 1:
manera recordándome que ya lo había perdonado y ofreciéndome rehabilitar a mi padre en el término de ocho días Esta última idea acabó por sobreponerse en mí a todas las demás y entonces sólo entonces le dije a Gutiérrez sin mirarlo -Por veinticinco mil duros causó usted la muerte y la deshonra de mi padre Cuánto dinero me pide usted ahora por su rehabilitación -A usted ninguno señor conde si no quiere dármelo -respondió Gutiérrez levantándose y yendo a ponerse detrás de mi butaca para librarme de su presencia- Soy pobre he perdido al juego aquella cantidad tengo familia en América pero a usted no le intereso nada sino aquello que sea su voluntad por devolverle como le voy a devolver o le devolverá el Senado el título de Conde y la secuestrada hacienda de su señor padre caudal que dicho sea entre nosotros asciende a más de ocho millones quién podrá pagarle a usted estos nuevos oficios caso que 

### Familia -- Positive

In [ ]:
df_pos_familia = df[
    (df['sentiment'] == 'Positive') &
    (df['text_window'].str.contains(r'\bfamilia\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Positive' and that contain the word 'familia': {len(df_pos_familia)}")


for i, text in enumerate(df_pos_familia['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")


Number of instances with sentiment 'Positive' and that contain the word 'familia': 5

Text 1:
oro que son más bonitas -dijo Tomasito a las que no creo yo llegue la bendición de la Iglesia puesto que el Sacerdote dice textualmente « Dignaos Señor bendecir estas palmas y ramos y no añade Estas cosas que parecen palmas Los niños se rieron y el abogado continuó Las palmas el laurel símbolo de la gloria y el olivo emblema de la paz son los vegetales que gozan del privilegio de ser bendecidos en tan fausto y solemne día pero volvamos a las plantas exóticas La caoba cedro doradillo ébano y todas las maderas preciosas proceden de árboles de América y Asia de entre los indígenas solamente el nogal se emplea para muebles de lujo pues a su dureza que le hace apreciable por la mucha duración que promete reúne la circunstancia de ser susceptible de un precioso pulimento El árbol del algodón de que me habéis hablado se cultiva en grande escala en los Estados Unidos de América Es de la familia de las

### "Guerra" -- Negative

In [ ]:
df_neg_guerra = df[
    (df['sentiment'] == 'Negative') &
    (df['text_window'].str.contains(r'\bguerra\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Negative' and that contain the word 'guerra': {len(df_neg_guerra)}")

# Imprimir cada ventana_texto individualmente
for i, text in enumerate(df_neg_guerra['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")


Number of instances with sentiment 'Negative' and that contain the word 'guerra': 6

Text 1:
Theodosio tragando saliva;- qué había de hacer con un Imperio desangrado por tantas crueles invasiones enfrente de las provincias más bravas del belicosísimo Occidente Galia Bretaña y España y con más amenazas de guerra que fronteras Esta vez Numisio se había dignado escuchar y no vaciló en desmentir al Emperador mientras se acercaba a él con los puños apretados como si tratara de agredirle -Dices eso a sabiendas de que no es verdad Gozaba el Imperio de una paz octaviana No te amenazaba guerra ninguna ni por el lado de las razas morenas del Nilo de la Arabia o del Asia Menor ni por el lado de las razas rubias que o estaban como siguen estando devoradas por la guerra civil -es el caso de los Persas- o absorbidas en las artes de la paz no hostilizadas por los hunnos ni solicitadas por ningún nuevo aspirante a rey -es el caso de los godos -Eso piensas tú -repuso Theodosio que iba recobrando su apl

### "Guerra" -- Positive

In [ ]:
df_pos_guerra = df[
    (df['sentiment'] == 'Positive') &
    (df['text_window'].str.contains(r'\bguerra\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Positive' and that contain the word 'guerra': {len(df_pos_guerra)}")

# Imprimir cada ventana_texto individualmente
for i, text in enumerate(df_pos_guerra['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")


Number of instances with sentiment 'Positive' and that contain the word 'guerra': 5

Text 1:
su propia madre Para Elena Antonio su padre era viudo y no le hablaba nunca de su madre por no renovar recuerdos tristes de otros tiempos ni abrir heridas del alma recuerdos y heridas que todavía destilaban sangre según las tristezas continuas del mulato cuyas pasiones se hallaban todas reunidas y concentradas en su hija Juntos habían recorrido toda Europa y gozado todos los esparcimientos propios de un viaje tan delicioso Juntos vivían los cuatro en una paz completa sin que hubiera objeto preferible a Elena para su cariño ni otro heredero a su inmensa fortuna Carolina estuvo adherida a América mientras vivió su esposo el caballero Jura Le era imposible dejarlo y cuidaba de él muchas veces con riesgo de la vida porque su locura llegaba con facilidad al furor y por consiguiente a la violencia Pero en cuanto Jura murió en cuanto pasó el duelo en cuanto guardó por un año entero el luto junto al ce

### "Enfermo" -- Positive

In [ ]:
df_pos_enfermo = df[
    (df['sentiment'] == 'Positive') &
    (df['text_window'].str.contains(r'\benfermo\b', case=False, na=False))
]


print(f"Number of instances with sentiment 'Positive' and that contain the word 'enfermo': {len(df_pos_enfermo)}")


for i, text in enumerate(df_pos_enfermo['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Positive' and that contain the word 'enfermo': 3

Text 1:
esta le prodigaba a prima noche las atenciones y cuidados que exige un enfermo Pablo le decía -Prima mía mi padre me ha leído aquel pasaje de nuestra historia cuando un hombre llamado Cristóbal Colón descubrió el Mundo Nuevo jamás visto por hombre alguno de Europa Aquel navegante abrió los ojos del mundo conocido para que viera otro más hermoso No puedo figurármelo a él sino como a un Teodoro Golfín y a la Europa como a un gran ciego para quien la América y sus maravillas fueron la luz Yo también he descubierto un Nuevo Mundo Tú eres mi América tú eres aquella primera isla hermosa donde puso su pie el navegante Faltole ver el continente con sus inmensos bosques y ríos A mí también me quedará por ver quizás lo más hermoso Después cayó en profunda meditación y al cabo de ella preguntó dónde está la Nela -No sé qué le pasa a esa pobre muchacha -dijo Florentina- No quiere verte sin duda -Es vergon

### "Enfermo" -- Negative

In [ ]:
df_neg_enfermo = df[
    (df['sentiment'] == 'Negative') &
    (df['text_window'].str.contains(r'\benfermo\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Negative' and that contain the word 'enfermo': {len(df_neg_enfermo)}")


for i, text in enumerate(df_neg_enfermo['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Negative' and that contain the word 'enfermo': 3

Text 1:
flor de tête revelaban una tristeza de lúbrico aburrido y enfermo Sus grandes bigotes grises adheridos en parte a las mejillas terrosas parecían un rabo de zorra -Vive le Sha -gritaron algunos y el landó custodiado por la guardia republicana y seguido por los del séquito imperial echó a andar hacia el Bosque paseo predilecto del monarca -Prepárese usted -dijo el doctor con cierta jovialidad- a oír toda una conferencia usted la ha pedido geográfico-histórica sobre la Persia -Je ne demande pas mieux -contestó Plutarco sonriendo -El antiguo imperio medo-persa -dijo el médico- estaba situado en la parte occidental del Asia Le limitaban por el Norte la cordillera del Cáucaso el mar Caspio y la Partia por el Este los montes de la India por el Sur el mar Eritreo el golfo Pérsico y la Arabia y por el Oeste el desierto de Libia el Mediterráneo el mar Egeo y el Ponto-Euxino El Éufrates dividía el imperi

## "Jefe" -- Positive

In [ ]:
df_pos_jefe = df[
    (df['sentiment'] == 'Positive') &
    (df['text_window'].str.contains(r'\bjefe\b', case=False, na=False))
]

print(f"Number of instances with sentiment 'Positive' and that contain the word 'jefe': {len(df_pos_jefe)}")


for i, text in enumerate(df_pos_jefe['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Positive' and that contain the word 'jefe': 5

Text 1:
marcha a saltos que terminó con una pantomima muy graciosa en que todos los saltarines hacían con la cara gestos muy semejantes a los del conejo cuando come En este extremo ninguno igualaba al jefe que es el inventor del juego y por esta razón se llama Sungo que quiere decir conejo Noté que de todas las figuras ésta era la que más agradaba al rey quien retrasó el silbido reglamentario y tuvo a los ejecutantes cerca de media hora moviendo la boca la nariz y las orejas En todos los pueblos hay un animal que simboliza la astucia en Asia el chacal en Europa la zorra En Maya no hay zorras ni chacales y el instinto popular cifra todos los rasgos de la astucia en el conejo cuyo fruncimiento constante de hocico contrastando con la impasibilidad de su mirada y la posición expectante de sus orejas ofrece cierto aire de picardía que nosotros los psicólogos europeos no hemos advertido Un artista como Sungo h

### "Jefe" -- Negative

In [ ]:
df_neg_jefe = df[
    (df['sentiment'] == 'Negative') &
    (df['text_window'].str.contains(r'\bjefe\b', case=False, na=False))
]

# Mostrar cuántas filas hay
print(f"Number of instances with sentiment 'Negative' and that contain the word 'jefe': {len(df_neg_jefe)}")

# Imprimir cada ventana_texto individualmente
for i, texto in enumerate(df_neg_jefe['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Negative' and that contain the word 'jefe': 3

Text 1:
muy bonita Conste que tú la encargaste -Yo sólo te hablaba de una caja -¿Y se puede saber para qué la quieres -Para guardar documentos -¿Importantes -Mucho -No serán cédulas porque ahí caben pocas -Son tus cartas -Pero chica guardarás todas las que te escribo -¿Me crees capaz de tirarlas -Pues yo rompo las tuyas en cuanto las leo -Está usted faltando a la verdad y a sabiendas -¡Caspitina -Las he encontrado todas -¿Dónde -En un secreto del pupitre no me acordaba -Merecías si lo merezco dámelo Y al año siguiente se dejaba convencer la madraza y me marchaba a Filipinas o a la Isla de Cuba Así pasa su existencia el Jefe de marina en esta época de paz para el Ejército y en que todas las luchas se reducen a cabildear por los pasillos del Ministerio o de las Cámaras y mover el personal a gusto de cuatro caciques Yo no he de referir estas miserias porque no deben conocerlas los profanos ni es posible rem

### "Cura" -- Negative

In [ ]:
df_neg_cura = df[
    (df['sentiment'] == 'Negative') &
    (df['text_window'].str.contains(r'\bcura\b', case=False, na=False))
]

# Mostrar cuántas filas hay
print(f"Number of instances with sentiment 'Negative' and that contain the word 'cura': {len(df_neg_cura)}")

# Imprimir cada ventana_texto individualmente
for i, text in enumerate(df_neg_cura['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Negative' and that contain the word 'cura': 1

Text 1:
una tardía pasión desesperada llena de sueños deliciosos de horror y lacerias Enfermó de fiebres y deliquios Una noche culpó a su hijo de la muerte de la madre y no quiso besarlo Después arrepentido levantose de la cama lo tomó en sus brazos cubriéndole de caricias y de lágrimas Sus violencias sus ternuras y su silencio atemorizaron a todos sentían un miedo ciego de infortunios que parecía que ya anticipaban su sombra sobre el hogar Una mañana pidió Agustín la galera dijo que iba a Valencia Y no volvió más Desde Génova escribió despidiéndose a punto de embarcarse para Filipinas No podía vivir en Serosca El reposo y el amor de su casa la amorosa compañía del músico la sonrisa de su hijo aquel sutil aroma del bien perdido y la emoción de vida de la muerta que al principio de su viudez le desgarraban le hirieron después con blandeza y quizá acabasen por ungirle las llagas y curarle como se cura un e

### Cura -- Positive

In [ ]:
df_pos_cura = df[
    (df['sentiment'] == 'Positive') &
    (df['text_window'].str.contains(r'\bcura\b', case=False, na=False))
]

# Mostrar cuántas filas hay
print(f"Number of instances with sentiment 'Positive' and that contain the word 'cura': {len(df_pos_cura)}")

# Imprimir cada ventana_texto individualmente
for i, text in enumerate(df_pos_cura['text_window'], 1):
    print(f"\nText {i}:\n{text}\n{'-'*60}")

Number of instances with sentiment 'Positive' and that contain the word 'cura': 1

Text 1:
y allí en presencia del propio confesor cura párroco de un pueblecillo próximo me dijo estas espantosas palabras -«Fabián me voy Tengo que dejarte solo sobre la tierra Lo manda Dios Ha llegado pues el caso de que te hable como se le habla a todo un hombre que eso serás desde mañana no obstante tu corta edad un hombre libre dueño de sus acciones sin nadie que lo aconseje y guíe por los mares de la vida Fabián hasta aquí has estado en la creencia de que tu padre mi difunto esposo fue un oscuro marino que murió en América dejándonos un modesto caudal Pero nada de esto es cierto Lo cierto es una cosa horrible que yo debo revelarte para que nunca te la enseñe el mundo por medio de crueles desvíos o sea para que jamás hagas imprudentes alardes de tu noble cuna que al cabo podrías conocer andando el tiempo aunque yo nada te contase Fabián mi marido fue el general don Álvaro Fernández de Lara conde de la